In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
import joblib

# Load dataset
df = pd.read_csv('customer_churn_data2(in).csv')

# Features and Target
selected_features = [
    'App Logins', 'Loans Accessed', 'Loans Taken',
    'Sentiment Score', 'Web Logins', 'Monthly Avg Balance',
    'Declined Txns', 'Overdraft Events', 'Tickets Raised'
]
target = 'Churned'

# 1️⃣ Extract Features and Target
X_raw = df[selected_features]
y = df[target]

# 2️⃣ Imputation
imputer = SimpleImputer(strategy='mean')
X_imputed = pd.DataFrame(imputer.fit_transform(X_raw), columns=selected_features)

# 3️⃣ Scaling
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X_imputed), columns=selected_features)

# 4️⃣ Split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, stratify=y, random_state=42
)

# 5️⃣ Model
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
model.fit(X_train, y_train)

# # 6️⃣ Save artifacts
# joblib.dump(model, 'xgb_churn_model2.pkl')
# joblib.dump(scaler, 'scaler.pkl')
# joblib.dump(imputer, 'imputer.pkl')
# Optional: Check accuracy
print("Training Accuracy:", model.score(X_train_scaled, y_train))
print("Test Accuracy:", model.score(X_test_scaled, y_test))


Training Accuracy: 0.922375
Test Accuracy: 0.8755


c:\Users\nroha\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [16:10:19] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [30]:
# ------------------ Predict Churn Probability ------------------
y_prob = model.predict_proba(X_test_scaled)[:, 1]

# ------------------ Assign Risk Tier ------------------
def assign_risk_tier(prob):
    if prob < 0.4:
        return "Low"
    elif 0.4 <= prob < 0.7:
        return "Medium"
    else:
        return "High"

risk_tiers = [assign_risk_tier(p) for p in y_prob]

# ------------------ Generate Recommendations (Based Only on Risk Tier) ------------------
def generate_recommendation(row):
    tier = row['Risk_Tier']
    actions = []

    if tier == 'Low':
        actions.append("Send monthly loyalty points")
    elif tier == 'Medium':
        actions.append("Offer $5 cashback for next renewal")
    elif tier == 'High':
        actions.append("Immediate support call + plan upgrade offer")

    return actions

# ------------------ Build Final Result DataFrame ------------------
final_result = X_test.copy()
final_result['Churn_Probability'] = y_prob
final_result['Risk_Tier'] = risk_tiers
final_result['Recommendations'] = final_result.apply(generate_recommendation, axis=1)

# ------------------ Show Final Dataset ------------------
print("\n📊 Final Result with Predictions and Recommendations:")
print(final_result[['Churn_Probability', 'Risk_Tier', 'Recommendations']])


📊 Final Result with Predictions and Recommendations:
      Churn_Probability Risk_Tier  \
5250           0.066108       Low   
336            0.651217    Medium   
262            0.016170       Low   
2208           0.000455       Low   
8442           0.015544       Low   
...                 ...       ...   
7846           0.942578      High   
8128           0.039300       Low   
9074           0.021378       Low   
8389           0.002375       Low   
1845           0.691480    Medium   

                                    Recommendations  
5250                  [Send monthly loyalty points]  
336            [Offer $5 cashback for next renewal]  
262                   [Send monthly loyalty points]  
2208                  [Send monthly loyalty points]  
8442                  [Send monthly loyalty points]  
...                                             ...  
7846  [Immediate support call + plan upgrade offer]  
8128                  [Send monthly loyalty points]  
9074           

In [31]:
joblib.dump(model, 'xgb_model.pkl')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(imputer, 'imputer.pkl')

['imputer.pkl']

In [24]:
final_result

,App Logins,Loans Accessed,Loans Taken,Sentiment Score,Web Logins,Monthly Avg Balance,Declined Txns,Overdraft Events,Tickets Raised,Churn_Probability,Risk_Tier,Top_Features,Recommendations
5250,10.0,2.0,1.0,0.793708,8.0,45719.80463,0.0,3.0,4.0,0.066108,Low,NaN,[Send monthly loyalty points]
336,12.0,0.0,5.0,0.599941,6.0,43957.82521,5.0,6.0,1.0,0.651217,Medium,"[Monthly Avg Balance, Declined Txns, App Logins]","[Send re-engagement email + free month trial, ..."
262,14.0,1.0,2.0,0.750811,5.0,60003.13638,3.0,0.0,2.0,0.016170,Low,"[Overdraft Events, Declined Txns, App Logins]","[Send re-engagement email + free month trial, ..."
2208,13.0,2.0,1.0,1.250448,8.0,68174.57524,4.0,0.0,3.0,0.000455,Low,NaN,[Send monthly loyalty points]
8442,14.0,1.0,2.0,0.427671,5.0,35189.47966,1.0,1.0,3.0,0.015544,Low,NaN,[Send monthly loyalty points]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7846,9.0,4.0,2.0,0.009352,4.0,46118.08028,3.0,1.0,3.0,0.942578,High,NaN,[Immediate support call + plan upgrade offer]
8128,8.0,1.0,2.0,0.469705,6.0,58564.32922,1.0,1.0,2.0,0.039300,Low,NaN,[Send monthly loyalty points]
9074,11.0,1.0,0.0,0.850200,10.0,27464.68166,2.0,4.0,1.0,0.021378,Low,NaN,[Send monthly loyalty points]
8389,18.0,0.0,2.0,0.066096,9.0,47760.24835,3.0,1.0,3.0,0.002375,Low,NaN,[Send monthly loyalty points]
